In [7]:
from bs4 import BeautifulSoup
import re
import gensim
from gensim.summarization.summarizer import summarize
from gensim.summarization.summarizer import summarize_corpus
import requests

In [1]:
def artist_album_list(artist_list):
    '''Takes a list of artists, returns list of their albums in order of input index (artist index)'''
    album_urls = []
    for i,_ in enumerate(artist_list):
        artist = str(artist_list[i])
        artist = re.sub(' ', '-', artist)
        #artist = str(artist)
        #artist = str(artist_list[i])
        base_artist_url = 'https://genius.com/artists/'
        artist_url = base_artist_url + artist
        #page = requests.get(artist_url)    
        #html = BeautifulSoup(page.text, "html.parser") this one also works
        r = requests.get(artist_url)
        soup = BeautifulSoup(r.content, 'lxml')
        for a in soup.find_all('a',{'class': 'vertical_album_card'}, ['href']):
            album_urls.append(a['href'])
    return album_urls

In [2]:
def album_song_list(album_urls):
    ''' Takes the list of albums and returns a list of songs in those albums'''
    song_urls = []
    for i,_ in enumerate(album_urls):
        r = requests.get(album_urls[i])
        soup = BeautifulSoup(r.content, 'lxml')
        for div in soup.find_all('div',{'class': 'chart_row chart_row--light_border chart_row--full_bleed_left chart_row--align_baseline chart_row--no_hover'}):
            if len(div.span.span.text) > 0: # The only thing that distinguishes between songs and extra links is the span #
                song_urls.append(div.a['href'])
    return song_urls

In [19]:
def lyrics_summarize(song_urls):
    song_info = []
    for i,_ in enumerate(song_urls):
        page = requests.get(song_urls[i])    
        html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

        # Song name
        try:
            song_name = html.find("h1", class_="header_with_cover_art-primary_info-title").get_text().encode('ascii','ignore').decode('ascii')
            artist = html.find("a",class_='header_with_cover_art-primary_info-primary_artist').get_text().encode('ascii','ignore').decode('ascii')
            album = html.find("a", class_="song_album-info-title").get_text().encode('ascii','ignore').decode('ascii')
            lyrics = html.find("div", class_="lyrics").get_text().encode('ascii','ignore').decode('ascii')
            lyrics = html.find("div", class_="lyrics").get_text().encode('ascii','ignore').decode('ascii')
            lyrics = re.sub('\[.*\]','',lyrics) # Remove [Verse] and [Bridge] stuff
            #lyrics = re.sub('\n',' ',lyrics)  # Remove gaps between verses
            song_info.append(song_name)
            song_info.append(artist)
            song_info.append(album)
            song_info.append(lyrics)
        except:
            pass
    return song_info      # unused

In [216]:
def trimmer(song_url, ratio=0.07):  
    song_info = []
    #lyrics_only = []
    page = requests.get(song_url)
    html = BeautifulSoup(page.text, "html.parser")
    #print(html)
    song_name = html.find("h1", class_="header_with_cover_art-primary_info-title").get_text().encode('ascii','ignore').decode('ascii')
    #print(song_name)
    artist = html.find("a",class_='header_with_cover_art-primary_info-primary_artist').get_text().encode('ascii','ignore').decode('ascii')
    #album = html.find("a", class_="song_album-info-title").get_text().encode('ascii','ignore').decode('ascii')
    lyrics = html.find("div", class_="lyrics").get_text().encode('ascii','ignore').decode('ascii')
    lyrics = html.find("div", class_="lyrics").get_text().encode('ascii','ignore').decode('ascii')
    lyrics = re.sub('\[.*\]','',lyrics) # Remove [Verse] and [Bridge] stuff
    #lyrics = re.sub('\n',' ',lyrics)  # Remove gaps between verses
    #song_info.append(song_name)
    #song_info.append(artist)
    #song_info.append(album)
    #song_info.append(lyrics)
    #print(song_info)
#     print("Song name: ", song_name)
#     print("Artist: ", artist)
#     print("")
#     print("Song Summary:")
    dirty = summarize(lyrics, ratio=ratio)
    clean = re.sub('\n', ' \n', dirty)
    song_info.append(song_name)
    song_info.append(artist)
    song_info.append(clean)
    return song_info
    

In [31]:
artist_list = ['mansionz']

mansionz = artist_album_list(artist_list)

mansionz_songs = album_song_list(mansionz)

mansionz = lyrics_summarize(mansionz_songs)

mansionz = mansionz[3::4]

In [110]:
print(summarize(mansionz[3], ratio=0.1))

Yeah, I'm sittin' in the goddamn 'Lac (Ya dig it)
Yeah, sittin' in the clean Impala (Ya dig it)
Because a lot of people don't wanna talk about Dennis Rodman anymore because they think he's a has-been
I said, "I don't give a fuck, man, I'ma go do Dennis Rodman"


In [94]:
path = 'https://genius.com/Migos-hoe-on-a-mission-lyrics'
trimmer(path, ratio=0.06)

Song name:  Hoe On A Mission
Artist:  Migos

Song Summary:
I put my hoe on a mission, she got to play her position
I got a thousand niggas, stay with some ammunition
Aint got no time for no playing no joking
Shitting on niggas, got diarrhea
Tryna steal, nigga is you for real?


In [103]:
path = 'https://genius.com/21-savage-slaughter-ya-daughter-lyrics'
trimmer(path, ratio=0.06)

Song name:  Slaughter Ya Daughter
Artist:  21 Savage

Song Summary:
Gon' pull up in, okay
'Cause a nigga like me be goddamn, rapping
At the same time lil' nigga out here, trapping
Fuck what your deal worth
Nigga, I look like Spielberg
She had a Honda (slaughter your daughter)


In [116]:
path = 'https://genius.com/Lil-uzi-vert-money-mitch-lyrics'
trimmer(path, ratio=0.07)
# Downfall is that if a verse is preceded by the same line, that line will get picked a lot

Song name:  Money Mitch
Artist:  Lil Uzi Vert

Song Summary:
Crib look like the Playboy mansion got sixteen bunnies
Yeah 'member they ain't love me, now these girls wanna hug me
Only said that 'cause their boyfriends listenin' but these girls wanna fuck me
Yeah, got the rari, saw the Wraith, run it
If I talk to your girl, man I swear you lucky
Huh, run it, run it, bitch, run it, yeah


In [217]:
path = 'https://genius.com/Adele-send-my-love-to-your-new-lover-lyrics'
tester = trimmer(path, ratio=0.07)
# Downfall is that if a verse is preceded by the same line, that line will get picked a lot

In [218]:
print(tester[2])

Mmm, told me you were ready 
Mmm, that was what you told me 
(Send my love to your new lover) 
(Send my love to your new lover)


In [205]:
print("test \n test")

test 
 test


In [123]:
import spotipy
import spotipy.oauth2

credentials = spotipy.oauth2.SpotifyClientCredentials('dd078c444ae142df817917fade581201','c67e8c5e778a45a591ac3c59f2417822')
spotify = spotipy.Spotify(client_credentials_manager=credentials)
print (spotify.audio_features(['4uLU6hMCjMI75M1A2tKUQC']))

[{'danceability': 0.721, 'energy': 0.939, 'key': 8, 'loudness': -11.823, 'mode': 1, 'speechiness': 0.0376, 'acousticness': 0.115, 'instrumentalness': 3.79e-05, 'liveness': 0.108, 'valence': 0.914, 'tempo': 113.309, 'type': 'audio_features', 'id': '4uLU6hMCjMI75M1A2tKUQC', 'uri': 'spotify:track:4uLU6hMCjMI75M1A2tKUQC', 'track_href': 'https://api.spotify.com/v1/tracks/4uLU6hMCjMI75M1A2tKUQC', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4uLU6hMCjMI75M1A2tKUQC', 'duration_ms': 213573, 'time_signature': 4}]


In [125]:
#import spotipy
credentials = spotipy.oauth2.SpotifyClientCredentials('dd078c444ae142df817917fade581201','c67e8c5e778a45a591ac3c59f2417822')
spotify = spotipy.Spotify(client_credentials_manager=credentials)

sp = spotipy
sp.trace = False

# find album by name
album = "Pestbringer"
results = sp.search(q = "album:" + album, type = "album")

# get the first album uri
album_id = results['albums']['items'][0]['uri']

# get album tracks
tracks = sp.album_tracks(album_id)
for track in tracks['items']:
    print(track['name'])

AttributeError: module 'spotipy' has no attribute 'search'

In [127]:
sp.search(q='artist:foo track:bar')

AttributeError: module 'spotipy' has no attribute 'search'

In [132]:
name = 'Migos'
results = spotify.search(q='artist:' + name, type='artist')
items = results['artists']['items']

In [133]:
items

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'},
  'followers': {'href': None, 'total': 4374146},
  'genres': ['pop', 'rap', 'trap music'],
  'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG',
  'id': '6oMuImdp5ZcFhWP0ESe6mG',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/29180f7f9c945e04de9b39cb5188f6a9171ae485',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/f31fe04e6bfd7d154cc6434cbec87483b8d2d46e',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/98b8cc9b24841b0f516dfc52e9228ff7f47041f2',
    'width': 160}],
  'name': 'Migos',
  'popularity': 95,
  'type': 'artist',
  'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3dn4XHZRNPUxC2UkepJmxy'},
  'followers': {'href': None, 'total': 14348},
  'genres': [],
  'href': 'https://api.spotify.com/v1/artists/3dn4XHZRNPUxC2UkepJmxy',
  'id': '3dn4XHZRN

In [147]:
results

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3AMigos&type=artist&offset=0&limit=10',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'},
    'followers': {'href': None, 'total': 4374146},
    'genres': ['pop', 'rap', 'trap music'],
    'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG',
    'id': '6oMuImdp5ZcFhWP0ESe6mG',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/29180f7f9c945e04de9b39cb5188f6a9171ae485',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/f31fe04e6bfd7d154cc6434cbec87483b8d2d46e',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/98b8cc9b24841b0f516dfc52e9228ff7f47041f2',
      'width': 160}],
    'name': 'Migos',
    'popularity': 95,
    'type': 'artist',
    'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/3dn4XHZRNPUxC2UkepJmxy

In [148]:
results['artists']['items'][1]['uri']

'spotify:artist:3dn4XHZRNPUxC2UkepJmxy'

In [158]:
results = spotify.search(q='YoungBoy Never Broke Again', limit=20)
for i, t in enumerate(results['tracks']['items']):
    print(' ', i, t['name'], t['id'])

  0 No Smoke 57IRaiAB4hBZu3gnNVZC0v
  1 Outside Today 5Qx9hngUBr30JFrNua7yp4
  2 Beast Mode (feat. PnB Rock & YoungBoy Never Broke Again) 1AH60dMaEfNjzDoUIFwIQi
  3 NBAYOUNGBOAT 6K2anECyrckidwf5wxS78Q
  4 Solar Eclipse 0Ofqw9JLatNDAXozUOynst
  5 Graffiti 6oLANSb6vq2T5tIg92FxCW
  6 Untouchable 6xL9MJ2yt39HPPZPyGkRy8
  7 Love Is Poison 3grLWp1ZsXJhpTc6PFS9s2
  8 Untouchable 4MofYf0f4ijlVV6elUW5S3
  9 Check Callin (feat. YoungBoy Never Broke Again) 64OFZXf5IvSBYX8dS8Wlaj
  10 In The Bank (feat. YoungBoy Never Broke Again) 0sEfomtr8d86aJgZxlfh3k
  11 GG 2DWdaFZYQyFFl4bFXC5DQi
  12 Dedicated 4wYiSiqO7VaG3J1i1sAd8R
  13 Doin Bad (feat. YoungBoy Never Broke Again) 0LPnvvNFTFg3pd71IlWs9n
  14 War With Us 627Ue4WpTs3P9DumoK3BVx
  15 You The One 6ChdMi0GZVAM6ENMwNvyjk
  16 Left Hand Right Hand 7e0o82uBiJZDdh0J6U3121
  17 Pleading The Fifth (feat. Quavo) 3LUPftNsM4KOk7kINY8keB
  18 No. 9 2fZ00UKwotKWzHPGdNtdK9
  19 38 Baby 35zkqY6kIGshmm8HhE3eSi


In [152]:
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Migos&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'},
       'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG',
       'id': '6oMuImdp5ZcFhWP0ESe6mG',
       'name': 'Migos',
       'type': 'artist',
       'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}],
     'available_markets': ['CA', 'MX', 'US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/7fd7SEK25VS3gJAUgSwL6y'},
     'href': 'https://api.spotify.com/v1/albums/7fd7SEK25VS3gJAUgSwL6y',
     'id': '7fd7SEK25VS3gJAUgSwL6y',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/79ef8fb6e772a47e3e045a747376dca7b900583e',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/dfc7c4f6f238bdd64f37c707e82a9b91cf3a0910',
       'width': 300},
      {'height': 64

In [159]:
not_rude = ['Ed Sheeran', 'Adele', 'Taylor Swift', 'Justin Timberlake', 'Bon Jovi']
rude = ['21 Savage', 'Migos', 'Lil Uzi Vert', 'Big TC', 'YoungBoy Never Broke Again']

In [166]:
results = []
for item in rude:
    results.append(spotify.search(q=item, limit=20))
    for i, t in enumerate(results['tracks']['items']):
        print(' ', i, t['name'], t['id'])

TypeError: list indices must be integers or slices, not str

In [168]:
results

[{'tracks': {'href': 'https://api.spotify.com/v1/search?query=21+Savage&type=track&offset=0&limit=20',
   'items': [{'album': {'album_type': 'single',
      'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4kYSro6naA4h99UJvo89HB'},
        'href': 'https://api.spotify.com/v1/artists/4kYSro6naA4h99UJvo89HB',
        'id': '4kYSro6naA4h99UJvo89HB',
        'name': 'Cardi B',
        'type': 'artist',
        'uri': 'spotify:artist:4kYSro6naA4h99UJvo89HB'},
       {'external_urls': {'spotify': 'https://open.spotify.com/artist/1URnnhqYAYcrqrcwql10ft'},
        'href': 'https://api.spotify.com/v1/artists/1URnnhqYAYcrqrcwql10ft',
        'id': '1URnnhqYAYcrqrcwql10ft',
        'name': '21 Savage',
        'type': 'artist',
        'uri': 'spotify:artist:1URnnhqYAYcrqrcwql10ft'}],
      'available_markets': ['AD',
       'AR',
       'AT',
       'AU',
       'BE',
       'BG',
       'BO',
       'BR',
       'CA',
       'CH',
       'CL',
       'CO',
       'CR',

In [181]:
results = spotify.search(q='Migos', limit=20)
for i, t in enumerate(results['tracks']['items']):
    print(' ', i, t['name'], t['id'])

  0 Stir Fry 2UVbBKQOdFAekPTRsnkzcf
  1 I Get The Bag (feat. Migos) 1XRgIKC5TPwo7nWGyKqgG0
  2 Narcos 0wbnC9AUenxp613TYaJsGK
  3 Solitaire (feat. Migos & Lil Yachty) 3DxZfFYJLwKDJtUwnLEhkL
  4 Solitaire (feat. Migos & Lil Yachty) 54pAgnuDbBs4xaE4Hhi3NX
  5 Walk It Talk It 6n4U3TlzUGhdSFbUUhTvLP
  6 Danger (with Migos & Marshmello) 0nhVrTiCGiGRCoZOJiWzm1
  7 Slide 6gpcs5eMhJwax4mIfKDYQk
  8 BBO (Bad Bitches Only) 4BHzHISX21PYwGXiBEUQ70
  9 Gang Gang 6RKkHI2fkmgZq7uQPPav8f
  10 Notice Me 3OnOomd79OjqSYHchCQ6ht
  11 MotorSport 4wFjTWCunQFKtukqrNijEt
  12 T-Shirt 7KOlJ92bu51cltsD9KU5I7
  13 Slippery (feat. Gucci Mane) 6p8NuHm8uCGnn2Dtbtf7zE
  14 Supastars 7yt6eiZQCkVFEzwGp4Pbor
  15 Bad and Boujee (feat. Lil Uzi Vert) 4Km5HrUvYTaSUfiSGPJeQR
  16 Get Right Witcha 76Y0gxtTxN0FyDCYh5qYQj
  17 Emoji A Chain 0G9rmRmQDFk9O2tzZLyRkc
  18 Fight Night 2n5gVJ9fzeX2SSWlLQuyS9
  19 Higher We Go - Intro 43dOwOq0mB4i6j4SpcSkqC


In [180]:
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Migos&type=track&offset=0&limit=30',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'},
       'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG',
       'id': '6oMuImdp5ZcFhWP0ESe6mG',
       'name': 'Migos',
       'type': 'artist',
       'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}],
     'available_markets': ['CA', 'MX', 'US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/7fd7SEK25VS3gJAUgSwL6y'},
     'href': 'https://api.spotify.com/v1/albums/7fd7SEK25VS3gJAUgSwL6y',
     'id': '7fd7SEK25VS3gJAUgSwL6y',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/79ef8fb6e772a47e3e045a747376dca7b900583e',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/dfc7c4f6f238bdd64f37c707e82a9b91cf3a0910',
       'width': 300},
      {'height': 64

In [182]:
features = spotify.audio_features('2UVbBKQOdFAekPTRsnkzcf')

In [200]:
features[0]['danceability']
features[0]['tempo']

181.967

In [187]:
features

[{'acousticness': 0.00299,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2UVbBKQOdFAekPTRsnkzcf',
  'danceability': 0.815,
  'duration_ms': 190288,
  'energy': 0.816,
  'id': '2UVbBKQOdFAekPTRsnkzcf',
  'instrumentalness': 0,
  'key': 2,
  'liveness': 0.159,
  'loudness': -5.474,
  'mode': 1,
  'speechiness': 0.269,
  'tempo': 181.967,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/2UVbBKQOdFAekPTRsnkzcf',
  'type': 'audio_features',
  'uri': 'spotify:track:2UVbBKQOdFAekPTRsnkzcf',
  'valence': 0.498}]